# RustyBot(rbot)のフレームワークスケルトン(real_run (testnet))

注：　Jyupiter上ではダウンロードなどのバックグランド処理中に止まってしまうことが多いです。コマンドライン版を動かしてください。

テストネットを使って本番実行を行います。

リアル時間なので、on_tickを１０秒毎に、on_tickを100イベント毎に出力します。

`on_init`, `on_tick`, `on_clock`, `on_update`を必要に応じて実装します。

参考：
* [backtest skelton](./skelton_back_test.ipynb)
* [forward skelton](./skelton_forward_test.ipynb)

In [ ]:
# rbotをインストールします。
! pip install --upgrade pip
! pip install rbot

In [ ]:
from rbot import time_string        # μsの時間を文字列に変換するユーティリティ関数です。

class SkeltonAgent:      # クラス名は任意です
    def __init__(self):
        """Botの初期化処理です。パラメータなどを設定するといいでしょう。利用しなくても構いません。
        """
        self.tick_count = 0         # on_tickが呼び出された回数をカウントします。
    
    def on_init(self, session):
        """Botの初期化処理。Botの初期化時に一度だけ呼ばれます。
        通常はsession.clock_interval_secを指定しon_clockの呼び出し間隔を設定します。
        Args:
            session: セッション情報（Botの初期化時用に渡されます）
        """
        session.clock_interval_sec = 10        # 10秒ごとにon_clockを呼び出します。

    
    def on_tick(self, session, side, price, size):
        """取引所からの全ての約定イベント毎に呼び出される処理です（高頻度で呼び出されます）
        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            side: 売買区分です。"Buy"または"Sell"が設定されます。
            price: 約定価格です。
            size: 約定数量です。
        """
        
        # on_tickは高頻度によびだされるので、100回に1回だけ内容をプリントします。
        if self.tick_count % 100 == 0:
            print("on_tick: ", side, price, size)

        self.tick_count += 1
    
    def on_clock(self, session, clock):
        """定期的にフレームワークから呼び出される処理です。
        session.clock_interval_secで指定した間隔で呼び出されます。

        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)        
            clock: 現在時刻です。エポック時間からのマイクロ秒で表されます。
        """
        # 現在の時刻をプリントします。
        print("on_clock: ", clock, ": ", time_string(clock))
    
    def on_update(self, session, updated_order):
        """自分の注文状態が変化した場合に呼び出される処理です。
        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)                
            session: セッション情報（市況情報の取得や注文するために利用します)        
            updated_order: 注文状態が変化した注文情報です。
        """
        # 注文状態が変化した注文情報をプリントします。オーダーを発行しない限り呼び出されません。
        print("on_update", updated_order)    

    

## RealRunの準備

最初にサーバのアクセスキーを設定してください。以下の環境変数を用います。

* Binance
    * BINANCE_API_KEY
    * BINANCE_API_SECRET

* Bybit
    * BYBIT_API_KEY
    * BYBIT_API_SECRET


In [ ]:
# Binanceマーケットを指定します。

from rbot import Binance
from rbot import BinanceConfig

binance_exchange = Binance(production=False)    # テストネットを利用します。

binance_exchange.enable_order_with_my_own_risk = True    # 自己責任で注文を出す設定にします。これがないと注文が出せません。

binance_market = binance_exchange.open_market(BinanceConfig.BTCUSDT)

In [ ]:
# Bybitマーケットを指定します。(binanceかbybitのどちらか一方を選択してください)
from rbot import Bybit
from rbot import BybitConfig

bybit_exchange = Bybit(production=False)   # テストネットを利用します。
bybit_exchange.enable_order_with_my_own_risk = True    # 自己責任で注文を出す設定にします。これがないと注文が出せません。

config = BybitConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
bybit_market = bybit_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [ ]:
exchange = bybit_exchange
market = bybit_market

In [ ]:
# 過去ログを１日分ダウンロードします。

market.download_archive(ndays=1, verbose=True)

# 更新系のオペレーションは１つのプロセスしかできません。他にプロセスがある場合はデッドロックする場合があります。
# また大量のファイルをダウンロードするため、ディスク容量に注意してください。


In [ ]:
# DBの場所は以下の場所にあります。古いバージョンのrbotと非互換が発生した場合などDBファイルを一旦削除してください。
market.file_name

## 本番実行

本番実行は、データをダウンロードする時間がかかり,jupyter上ではうまく動かない場合が多いです。その場合は`skeltom_realrun.py`のほうをコマンドラインでうごかしてください。途中経過が表示されるので止まっているのか、動いているのか判別できます。

本番実行の場合、アーカイブデータとリアルタイムデータの間のデータをすべてダウンロードします。
* Binanceの場合
    RESTでダウンロードするのて時間がかかります。
* Bybitの場合
    アーカイブ以降は１分足のデータしかないので、１分足以下のOHLCVは作成しても正しくデータが反映されません。

In [ ]:
from rbot import Runner
from rbot import NOW, DAYS

from rbot import init_debug_log
init_debug_log()

agent = SkeltonAgent()
runner = Runner()

session = runner.real_run(
                exchange=exchange,
                market=market,
                agent=agent, 
                execute_time = 90,  # 90秒間実行します。指定しないと永久に実行します。
                verbose=True,
                log_file="skelton_bot.log"
            )

session

In [ ]:
exchange.account

# ここからは別プロセスで行う

本番は終了することが通常ない。
そのため、ログファイルを別途読み込んで分析する。
以下読み込み方のサンプル

In [ ]:
from rbot import Logger

log = Logger()
log.restore("skelton_bot.log")


In [ ]:
# ファイルから読み込んだあとは通常どおり。

log.orders

In [ ]:
log['indicators']

In [ ]:
# テストネットのIDはアーカイブと連続していないため不整合が発生します。次回のために削除しておきます。
#  アーカイブ以外、WSやRESTからのデータを削除します
market.expire_unfix_data(force=True)  

from time sleep
sleep(10)